In [0]:
from pyspark.ml.regression import RandomForestRegressionModel

model_path = "/Volumes/walmart_cat/gold/model_artifacts/rf_sales_model"
rf_model = RandomForestRegressionModel.load(model_path)


In [0]:
gold_df = spark.table("walmart_cat.gold.sales_features")


In [0]:
from pyspark.sql.functions import expr

numeric_cols = [
    "Temperature",
    "Fuel_Price",
    "MarkDown1",
    "MarkDown2",
    "MarkDown3",
    "MarkDown4",
    "MarkDown5",
    "CPI",
    "Unemployment",
    "Size",
    "year",
    "month",
    "week_of_year",
    "lag_1_week",
    "lag_2_week",
    "lag_4_week",
    "rolling_avg_4w"
]

clean_df = gold_df
for c in numeric_cols:
    clean_df = clean_df.withColumn(c, expr(f"try_cast({c} as double)"))

clean_df = clean_df.fillna(0)


In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=numeric_cols,
    outputCol="features",
    handleInvalid="skip"
)

prediction_input = assembler.transform(clean_df)


In [0]:
predictions = rf_model.transform(prediction_input)


In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_rmse = RegressionEvaluator(
    labelCol="Weekly_Sales",
    predictionCol="prediction",
    metricName="rmse"
)

evaluator_mae = RegressionEvaluator(
    labelCol="Weekly_Sales",
    predictionCol="prediction",
    metricName="mae"
)

rmse = evaluator_rmse.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")


RMSE: 27680.94423043529
MAE: 15863.599394595341


In [0]:
predictions.select(
    "Store",
    "Dept",
    "Date",
    "Weekly_Sales",
    "prediction"
).write.mode("overwrite").saveAsTable(
    "walmart_cat.gold.sales_predictions"
)


### Actual vs Predicted (time series)

In [0]:
%sql
SELECT
  Date,
  SUM(Weekly_Sales) AS actual_sales,
  SUM(prediction) AS predicted_sales
FROM walmart_cat.gold.sales_predictions
GROUP BY Date
ORDER BY Date;


Date,actual_sales,predicted_sales
2010-03-05,4.686660362E7,129861.99012411454
2010-03-12,4.5924557659999974E7,130541.8958315706
2010-03-19,4.4988556200000055E7,131221.80153902667
2010-03-26,4.413210203000004E7,131218.93703270121
2010-04-02,5.0422441930000044E7,132083.01543513767
2010-04-09,4.736427148000003E7,132536.28590677504
2010-04-16,4.5182413210000016E7,132581.61295393878
2010-04-23,4.4733294660000026E7,131856.38019931898
2010-04-30,4.370488226999987E7,131493.7638220091
2010-05-07,4.850128718999991E7,131584.41791633656


Databricks visualization. Run in Databricks to view.

### Top 5 Stores by Sales

In [0]:
%sql
SELECT
  Store,
  SUM(Weekly_Sales) AS total_sales
FROM walmart_cat.gold.sales_predictions
GROUP BY Store
ORDER BY total_sales DESC
LIMIT 5;


Store,total_sales
20,2.928240981100001E8
4,2.9124301952999985E8
14,2.8037005773999894E8
13,2.787291800099988E8
2,2.6711588401000062E8


Databricks visualization. Run in Databricks to view.

### Error by Store

In [0]:
%sql
SELECT
  Store,
  AVG(ABS(Weekly_Sales - prediction)) AS avg_error
FROM walmart_cat.gold.sales_predictions
GROUP BY Store
ORDER BY avg_error DESC;


Store,avg_error
20,29324.841469464453
4,28995.794210743956
14,28567.30442465441
13,27163.71686832578
2,26675.057958792782
10,26072.926141578228
27,24667.118285609395
6,21755.39768414828
1,21560.557837077326
39,20950.853303434713


Databricks visualization. Run in Databricks to view.

Lag features and rolling averages were the strongest predictors, showing that Walmart sales are highly dependent on historical demand patterns. External factors like temperature and unemployment contributed less compared to temporal features.